### Data Visualization - Ideal vs Encountered

In [2]:
#dataframes
import pandas as pd
import h5py

#suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#math
import numpy as np
import math as m
from scipy.spatial.distance import cdist
pd.TimeSeries = pd.Series 

#plots
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar
import figurefirst
from figurefirst import FigureLayout,mpl_functions
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter

#performance
import time 

In [3]:
def calc(desired_amount):
    return desired_amount/((100-5.5)/100)

In [7]:
((calc(400))-400 )/400

0.05820105820105823

### Load Data

In [2]:
dir = '~/Documents/Myfiles/DataAnalysis/data/Sprints/LowRes/'
# dir_save = '../../../Research/Images/container_odor/'
df = pd.DataFrame()
df = pd.read_hdf(dir+'NotWindy.h5')
dt= df.master_time[1]-df.master_time[0]

In [3]:
df["nfs"]=df.nearest_from_streakline
for i in range(len(df)):
        
    if(df.ysrc[i] < 0):
        df.nfs[i]=df.nfs[i]*-1
    

<ipython-input-3-e1ce2d7d1d16>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.nfs[i]=df.nfs[i]*-1


In [4]:
df["distance_from_source_a"]=np.sqrt(df.distance_from_source**2 - df.nearest_from_streakline**2)

In [5]:
df = df[df['time'].between(100,300)]
# df = df[df['time']>=2000]

In [6]:
df.reset_index(inplace=True, drop=True) 

In [7]:
len(df)

1500

### Prepare Data

In [7]:
def streakline_container(eastwest,northsouth):

    for i in range((len(eastwest))-1,-1, -1): 
        eastwest = np.resize(np.array([eastwest-df.corrected_u[i]*dt]),(1,i)).flatten() 
        northsouth = np.resize(np.array([northsouth-df.corrected_v[i]*dt]),(1,i)).flatten()
        x.loc[i]=np.pad(eastwest, ((len(df)-len(eastwest)),0),'constant', constant_values=(0))
        y.loc[i]=np.pad(northsouth, ((len(df)-len(northsouth)),0),'constant', constant_values=(0))

def streakline_calculation(df):
    et = [np.sum(df.corrected_u[j:])*dt for j in range(0,len(df))]
    nt = [np.sum(df.corrected_v[j:])*dt for j in range(0,len(df))]
    return et, nt

def prepare_df(df):
    df.pop(df.columns[0])
    strings=[]
    for i in range(len(df)-1,0,-1):
        strings.append("p"+str(i))
    df.columns=strings
    df.reset_index(inplace=True, drop=True) 
    return df

def c_bar(ax, var):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.08)
    cbar=f.colorbar(var, cax=cax, orientation = 'vertical', ticks=[0, 10])
    cbar.ax.set_yticklabels(['0', '10'])
    cbar.ax.set_ylabel('Odor(V)', rotation=-90, labelpad=-5)

In [8]:
et, nt = streakline_calculation(df)

In [9]:
strings=[]
for i in range(len(df)):
    strings.append("p"+str(i))

x=pd.DataFrame(columns=strings,index=range(len(df)))
y=pd.DataFrame(columns=strings,index=range(len(df)))
streakline_container(et,nt)
x=prepare_df(x)
y=prepare_df(y)

In [10]:
# dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/LowRes/'
# x.to_hdf(dir+'LWsample.h5', key='x', mode='w')
# y.to_hdf(dir+'Lwsample.h5', key='y', mode='w')

# # x=pd.read_hdf(dir+'LWx4500-5000.h5')
# # y=pd.read_hdf(dir+'Lwy4500-5000.h5')

In [10]:
dir_save = '../../Figure/sampleDirectionChange/'

In [14]:
# count = 0
tail=300

for i in range(0,len(df)):   
    f = plt.figure(figsize =(8,5))
    ax = plt.axes (xlim=(-16,12), ylim=(-5,25))
    ax.plot(0,0, 'x', label="Source")
    area = (np.arange(start = i, stop = 0 , step = -1))**2 #area
    ax.scatter(x.loc[i][x.loc[i]!=0],y.loc[i][y.loc[i]!=0], 
               c='#FFA500', s=np.sqrt(area),
               label="Streakline Derived\nfrom Wind", edgecolors='none',)
    
    ax.plot(df.xsrc[i],df.ysrc[i],marker='*', markersize=11, c='green', label='Sensor Current\nPosition')
    
    if (i>tail):          
        var = ax.scatter(df.xsrc[(i-tail):i],df.ysrc[(i-tail):i], c = df.odor[(i-tail):i], 
                   cmap = 'inferno_r', vmin =0 , vmax = 10, 
                   s =12)
        c_bar(ax, var)
    else:
        var = ax.scatter(df.xsrc[:i],df.ysrc[:i], c = df.odor[:i],
                         cmap = 'inferno_r', vmin =0 , vmax = 10, s =12 )       
        c_bar(ax, var)
   
    ax.set_xlabel('Longitude, m')
    ax.set_ylabel('Latitude, m')

    ax.set_xticks([-15,0,15])
    ax.set_yticks([-5,10,25])
    
    f.tight_layout(pad=1)
    figurefirst.mpl_functions.set_fontsize(f, 22)
    ax.text(-15.5, -4.2,"time= " + str('{:.4g}'.format(df.time[i]))+ " secs", 
             fontsize=12)
    lgnd=ax.legend(edgecolor='black', framealpha=1, loc='upper right')
    lgnd.legendHandles[1]._sizes = [15]
    lgnd.legendHandles[2]._sizes = [35]
    
    f.savefig(dir_save + "plot" + str(i) + ".jpg", dpi=300, bbox_inches = "tight")
    plt.close()

In [196]:
# tail=300
# for i in range(1500,1510, 1):   
#     f, (ax,ax1) = plt.subplots(2,1,figsize =(10,12))
#     ax.set_xlim(-16,12)
#     ax.set_ylim(-5,25)
#     ax.plot(0,0, 'x', label="Source")
#     area = (np.arange(start = i, stop = 0 , step = -1))**3 #area
#     ax.scatter(x.loc[i][x.loc[i]!=0],y.loc[i][y.loc[i]!=0], 
#                c='#FFA500', s=np.sqrt(area),
#                label="Streakline Derived\nfrom Wind", edgecolors='none',)
#     ax.plot(df.xsrc[i],df.ysrc[i],marker='*', markersize=11, c='green', label='Sensor Current\nPosition')
    
#     if (i>tail):          
#         var = ax.scatter(df.xsrc[(i-tail):i],df.ysrc[(i-tail):i], c = df.odor[(i-tail):i], 
#                    cmap = 'inferno', vmin =0 , vmax = 10, 
#                    s =12)
#         c_bar(ax, var)
    
#     else:
#         var = ax.scatter(df.xsrc[:i],df.ysrc[:i], c = df.odor[:i],
#                          cmap = 'inferno', vmin =0 , vmax = 10, 
#                    s =12 )       
#         c_bar(ax, var)
   
#     ax.set_xlabel('Longitude, m')
#     ax.set_ylabel('Latitude, m')

#     ax.set_xticks([-15,0,15])
#     ax.set_yticks([-5,10,25])
    
#     f.tight_layout(pad=2)
#     figurefirst.mpl_functions.set_fontsize(f, 22)
#     ax.text(-15.5, -4.2,"time= " + str('{:.4g}'.format(df.time[i]))+ " secs", 
#              fontsize=12)
#     lgnd=ax.legend(edgecolor='black', framealpha=1, bbox_to_anchor=(1,1))
#     lgnd.legendHandles[]._sizes = [35]
#     lgnd.legendHandles[]._sizes = [35]
    
#     f.savefig(dir_save + "plot" + str(i) + ".jpg", dpi=300, bbox_inches = "tight")
#     plt.close()

In [19]:
# eastwest = [np.sum(df.corrected_u[j:])*dt for j in range(0,len(df))]
# northsouth = [np.sum(df.corrected_v[j:])*dt for j in range(0,len(df))]

In [52]:
# count = 0
# N=2500

# #avoiding the last particle
# for i in range((len(eastwest))-1,0, -1):   
#     fig = plt.figure()
#     fig.suptitle('Radius time**0.5*0.01 - Run03_Set05_Small', fontsize =14)
    
#     ax = plt.axes (xlim=(-8,15), ylim=(-2,30))
#     ax.set_xlabel('Longitude (meters)')
#     ax.set_ylabel('Latitude(meters)')
    
#     #TODO: Ignoring 0th point
    
#     eastwest = np.resize(np.array([eastwest-df.corrected_u[i]*dt]),(1,i)).flatten() 
#     northsouth = np.resize(np.array([northsouth-df.corrected_v[i]*dt]),(1,i)).flatten()

#     area = np.arange(start = i, stop = 0, step = -1)**2*0.04*m.pi #area
    
#     ax.scatter(eastwest, northsouth, c='#FFA500', alpha = 0.3, s=(area/4))
    
#     if (count<2499):          
#         ax.scatter(df.xsrc[N:i],df.ysrc[N:i], c = df.odor[N:i], cmap = 'inferno', vmin =0 , vmax = 13, s =12)
#         N=N-1
        
#     else:
#         ax.scatter(df.xsrc[:i],df.ysrc[:i], c = df.odor[:i],cmap = 'inferno', vmin =0 , vmax = 13, s =12 )

#     count+=1
    
#     fig.savefig(dir_save + "plot" + str(i) + ".jpg")
#     plt.close()

### Imagewise Animation Odor_Expected vs Time

In [22]:
dir_save = '../../../Research/Images/container_wind/'

In [23]:
count = 0
N=0
for i in range(0,len(df)):
    
    f, ax = plt.subplots(1,1, figsize =(16,8))
    ax.set_xlim(-1,25)
    ax.set_ylim(-1,12)
    ax.axhline(0, color = 'red')
    ax.plot(0,0,'o', markersize = 20)
    
    if (count>500):          
        ax.scatter(df.distance_from_source[N:i],df.nearest_from_streakline[N:i], c = df.odor[N:i], cmap = 'inferno', vmin =0 , vmax = 13, s =12)
        N+=1
    else:
        ax.scatter(df.distance_from_source[:i],df.nearest_from_streakline[:i], c = df.odor[:i],cmap = 'inferno', vmin =0 , vmax = 13, s =12 )
    
    count+=1
    ax.set_xlabel('Distance from source to sensor(m)' , fontsize = 15)
    ax.set_ylabel('Shotest Distance \n Sensor from Streakline (m)' , fontsize = 15)
    figurefirst.mpl_functions.set_fontsize(f, 15)
    f.savefig(dir_save + "plot" + str(i) + ".jpg")
    plt.close()

### Animation using Matplotlib FuncAnimation

In [ ]:
# f , (ax1,ax2) = plt.subplots(2,1, figsize=(8,12)) 

# n=500
# def animate(i):
#     ax1.cla()
# #     ax2.cla()
# #     i=20*i
  
# #     area = (np.arange(start = len(ew[i]), stop = 0, step = -1)*0.01)**2*4*m.pi #area
# #     ax1.scatter(ew[i], ns[i], c='#FFA500', alpha = 0.3, s=np.sqrt(area), label="Expected Odor")
    
# #     if (i>500):          
# #         ax1.scatter(df.xsrc.iloc[(n):i],df.ysrc.iloc[(n):i], 
# #                     c = df.odor.iloc[(i-n):i], 
# #                     cmap = 'viridis', vmin =0 , vmax = 10, s =12)
# #         n+=n
# #     else:
# #         ax1.scatter(df.xsrc[:i],df.ysrc[:i], 
# #                     c = df.odor.iloc[:i], 
# #                     cmap = 'viridis', vmin =0 , vmax = 10, s =12)


#     if (i>500):          
#         ax2.scatter(df.distance_from_source_a.iloc[(i-n):i],df.nfs.iloc[(i-n):i], 
#                    c = df.odor.iloc[(i-n):i], 
#                    cmap = 'viridis', vmin =0 , vmax = 10, s =12)
# #         N+=1
#     else:
#         ax2.scatter(df.distance_from_source_a.iloc[:i],df.nfs.iloc[:i], c = df.odor.iloc[:i],
#                    cmap = 'viridis', vmin =0 , vmax = 10, s =12 )
        
# #     mpl_functions.adjust_spines(ax1,['left','right','bottom','top'],spine_locations={},
# #                             smart_bounds=True, xticks=[-15,0,10],yticks=[-5,10,20],linewidth=1)
#     mpl_functions.adjust_spines(ax2,['left','right','bottom','top'],spine_locations={},
#                             smart_bounds=True, xticks=[0,25,50],yticks=[-25,0,25],linewidth=1)

# #     ax1.set_xlabel('Longitude(meters)')
# #     ax1.set_ylabel('Latitude(meters)')
# #     ax2.set_xlabel("Distance Along Streakline")
#     ax2.set_ylabel("Nearest from Streakline")
#     f.tight_layout(pad=1)
#     figurefirst.mpl_functions.set_fontsize(f, 22)
    
# anim = FuncAnimation(f, animate, interval=10, frames=(len(df)-1), blit=False)
# writervideo = FFMpegWriter(fps=500)
# anim.save('../../Figure/test.mp4', writer=writervideo)
# HTML(anim.to_html5_video())